# DBSCAN
This notebook is split into three sections with one for every algorithm.
Essentially, the idea is to cluster movies and users so that we can
recommend the entire cluster they belong to, for both users and movies.

Compared to the other algorithms, this means that we will not be able
to give exactly how much they like a given movie, but just give them
the cluster they reside in.

## Initialise PySpark and data

In [26]:
from pyspark.ml.linalg import DenseVector
from pyspark.mllib.random import RandomRDDs
import pyspark.sql
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from sklearn.cluster import DBSCAN
import os
import numpy as np

from density.slides_dbscan import my_DBSCAN

if os.path.basename(os.getcwd()) == 'density':
    print("Current dir is", os.getcwd())
    print("Changing dir to be in root")
    os.chdir('..')
    print('now in', os.getcwd())

from proposal.useful_tools import pickle_manager

SPARK_CONF = SparkConf()
SPARK_CONF.set("spark.driver.memory", "10g")
SPARK_CONF.set("spark.cores.max", "4")
SPARK_CONF.set("spark.executor.heartbeatInterval", "3600")
SPARK_CONF.setAppName("word2vec")

SPARK_CONTEXT = SparkContext.getOrCreate(SPARK_CONF)
SPARK = SQLContext(SPARK_CONTEXT)

In [ ]:
# Load data: TODO
# Assume that I can do this
MOVIES: pyspark.sql.DataFrame = 1  # data frame
USERS: pyspark.sql.DataFrame = 1  # data frame

# Just so that we have something lets just go ahead and do this
size = 80000
np.random.seed(42)
MOVIES_SIMILARITY_MATRIX = pickle_manager.load_pickle('pickles/simil_matrix.pickle.lz4')
USERS_SIMILARITY_MATRIX = pickle_manager.load_pickle('pickles/simil_matrix.pickle.lz4')

# To keep things fair, initialise the necessary parameters right at the start
MOVIES_RADIUS = 0.001
MOVIES_MINIMUM_POINTS = 4

USERS_RADIUS = 0.001
USERS_MINIMUM_POINTS = 4

In [ ]:
print(MOVIES_SIMILARITY_MATRIX)

## Scikit DBSCAN
[Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN)
### Implementation

In [ ]:
scikit_movies_clustering = DBSCAN(
    eps=MOVIES_RADIUS, min_samples=MOVIES_MINIMUM_POINTS, metric='precomputed', n_jobs=-1
).fit(MOVIES_SIMILARITY_MATRIX)

scikit_users_clustering = DBSCAN(
    eps=USERS_RADIUS, min_samples=USERS_MINIMUM_POINTS, metric='precomputed', n_jobs=-1
).fit(MOVIES_SIMILARITY_MATRIX).fit(USERS_SIMILARITY_MATRIX)

In [ ]:
print(set(scikit_movies_clustering.labels_))
print(set(scikit_users_clustering.labels_))

## PyClustering DBSCAN
### [Documentation](https://pyclustering.github.io/docs/0.10.1/html/d2/d42/classpyclustering_1_1cluster_1_1dbscan_1_1dbscan.html#details)

In [ ]:
from pyclustering.cluster.dbscan import dbscan

pyclustering_ = dbscan(
    MOVIES_SIMILARITY_MATRIX, MOVIES_RADIUS, MOVIES_MINIMUM_POINTS
)

dbscan_instance = dbscan(
    USERS_SIMILARITY_MATRIX, USERS_RADIUS, USERS_MINIMUM_POINTS
)

## Slides
### Implementation
Since we made a class that just inherits the scikit dbscan and replaces the fit function, we should
just be able to do the same process here as the scikit section

In [ ]:
my_dbscan_movies_clustering = my_DBSCAN(
    eps=MOVIES_RADIUS, min_samples=MOVIES_MINIMUM_POINTS, metric='precomputed', n_jobs=-1
).fit(MOVIES_SIMILARITY_MATRIX)

my_dbscan_users_clustering = my_DBSCAN(
    eps=USERS_RADIUS, min_samples=USERS_MINIMUM_POINTS, metric='precomputed', n_jobs=-1
).fit(USERS_SIMILARITY_MATRIX)

In [ ]:
print(set(my_dbscan_movies_clustering.labels_))
print(set(my_dbscan_users_clustering.labels_))

## Evaluation